In [ ]:
!pip install -q tensorflow-recommenders

In [ ]:
import tensorflow_recommenders as tfrs
print(f"TensorFlow Recommenders version: {tfrs.__version__}")

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the data (you've already done this part)
train = pd.read_csv("/kaggle/input/playground-series-s5e2/train.csv")
train_extra = pd.read_csv("/kaggle/input/playground-series-s5e2/training_extra.csv")

In [ ]:
# Combine datasets for more training data
all_train = pd.concat([train, train_extra], ignore_index=True)
print(all_train.shape)
all_train[:10]

In [ ]:
all_train.isnull().sum()

In [ ]:
display(all_train.drop(columns=["Price"]).describe())

In [ ]:
sns.histplot(all_train["Weight Capacity (kg)"])

In [ ]:
display(all_train.Price.describe().to_frame().T)

sns.histplot(all_train.Price)

In [ ]:
# First, handle missing values
if all_train['Weight Capacity (kg)'].isnull().any():
    # Impute with the median
    median_weight = all_train['Weight Capacity (kg)'].median()
    all_train['Weight Capacity (kg)'].fillna(median_weight, inplace=True)

# Round to nearest integer to capture the spikes
all_train['Weight Capacity Int'] = all_train['Weight Capacity (kg)'].round().astype(int)

In [ ]:
categorical_features = ['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color']
numerical_features = ['Compartments', 'Weight Capacity Int']
target = 'Price'



In [ ]:
X = all_train.drop(['id', target, 'Weight Capacity (kg)'], axis=1)
y = all_train[target]

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create preprocessing pipelines for categorical and numerical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Combine preprocessors
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocess the data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)

In [ ]:

# Get the input dimensions for the model
input_dim = X_train_preprocessed.shape[1]

In [ ]:
def build_model(input_dim):
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)  # Output layer for regression
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='mean_squared_error',
        metrics=['mean_absolute_error', 'mean_squared_error']
    )
    
    return model


def build_dcn_model(input_dim, num_cross_layers=2, projection_dim=None, activity_regularizer=1e-5, kernel_regularizeer=1e-3):
    """
    Build a Deep & Cross Network model using TFRS implementation
    
    Args:
        input_dim: Dimension of input features
        num_cross_layers: Number of cross layers to use
        projection_dim: If not None, uses low-rank approximation for cross layers
    """
    # Input layer
    inputs = keras.Input(shape=(input_dim,))
    
    # Cross Network
    x_cross = inputs
    for _ in range(num_cross_layers):
        # Using the TFRS implementation of Cross layer
        x_cross = tfrs.layers.dcn.Cross(
            projection_dim=projection_dim,
            diag_scale=0.1,  # Small value to improve stability
            kernel_regularizer=keras.regularizers.l2(kernel_regularizeer),
            bias_regularizer=keras.regularizers.l2(kernel_regularizeer)
        )(x_cross)
    
    # Deep Net
    x_deep = keras.layers.Dense(128, 
                               activation='relu',
                               kernel_regularizer=keras.regularizers.l2(kernel_regularizeer),
                               activity_regularizer=keras.regularizers.l1(activity_regularizer))(inputs)
    x_deep = keras.layers.BatchNormalization()(x_deep)
    x_deep = keras.layers.Dropout(0.5)(x_deep)
    
    x_deep = keras.layers.Dense(64, 
                               activation='relu',
                               kernel_regularizer=keras.regularizers.l2(kernel_regularizeer))(x_deep)
    x_deep = keras.layers.BatchNormalization()(x_deep)
    x_deep = keras.layers.Dropout(0.4)(x_deep)
    
    x_deep = keras.layers.Dense(32, 
                               activation='relu',
                               kernel_regularizer=keras.regularizers.l2(kernel_regularizeer))(x_deep)
    x_deep = keras.layers.BatchNormalization()(x_deep)
    x_deep = keras.layers.Dropout(0.3)(x_deep)
    
    # Combine the two networks
    combined = keras.layers.Concatenate()([x_cross, x_deep])
    
    combined = keras.layers.Dense(32, 
                                 activation='relu',
                                 kernel_regularizer=keras.regularizers.l2(kernel_regularizeer))(combined)
    combined = keras.layers.Dropout(0.3)(combined)
    
    # Output layer
    outputs = keras.layers.Dense(1, kernel_regularizer=keras.regularizers.l2(kernel_regularizeer))(combined)
    
    # Create the model
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    optimizer = keras.optimizers.Adam(
        learning_rate=0.001,
        clipnorm=1.0,  # Clip gradients by norm
        amsgrad=True
    )
    
    # Compile
    model.compile(
        optimizer=optimizer,
        loss='mean_squared_error',
        metrics=['mean_absolute_error', "mean_squared_error"]
    )
    
    return model

# Create the model (standard)
# model = build_model(input_dim)
# Create model, DCN
model = build_dcn_model(input_dim)

# Define callbacks for training
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_mean_squared_error',
    mode='min',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.0001
)

weight_decay_callback = keras.callbacks.LearningRateScheduler(
    lambda epoch, lr: lr * 0.99  # Slight decay per epoch
)

In [ ]:
history = model.fit(
    X_train_preprocessed, y_train,
    epochs=30,
    batch_size=1024,
    # steps_per_epoch=1000, # comment for full data
    # validation_steps=1000,
    validation_data=(X_val_preprocessed, y_val),
    callbacks=[early_stopping, reduce_lr, weight_decay_callback,],
    verbose=1
)

In [ ]:
# Function to make predictions on test data
def predict_price(model, preprocessor, new_data):
    # We need to apply the same preprocessing to test data
    if 'Weight Capacity (kg)' in new_data.columns:
        # Handle missing values
        if new_data['Weight Capacity (kg)'].isnull().any():
            median_weight = new_data['Weight Capacity (kg)'].median()
            new_data['Weight Capacity (kg)'].fillna(median_weight, inplace=True)
        
        # Round to nearest integer
        new_data['Weight Capacity Int'] = new_data['Weight Capacity (kg)'].round().astype(int)
        
        # Drop the columns we don't need
        new_data = new_data.drop(['Weight Capacity (kg)'], axis=1)
    
    # Transform the data and predict
    new_data_preprocessed = preprocessor.transform(new_data)
    predictions = model.predict(new_data_preprocessed)
    return predictions


In [ ]:
test = pd.read_csv("/kaggle/input/playground-series-s5e2/test.csv")
print("Test shape", test.shape )
test[:5]

In [ ]:
preds = predict_price(model, preprocessor, test)

In [ ]:
sub = pd.read_csv("/kaggle/input/playground-series-s5e2/sample_submission.csv")
sub.Price = preds
sub.to_csv(f"submission.csv",index=False)
sub.head()